# Daten splitten

Einlesen der CSV-Rohdaten in pandas df

In [1]:
import pandas as pd
df = pd.read_csv('HABV_97.2_BöV-Mitgliederakten.csv', sep=';', index_col='Nr')

Das Feld Laufzeit wird in zwei Felder 'lz_beginn' und 'lz_beginn' gesplittet.

In [2]:
df['lz_beginn'] = df['Laufzeit'].str.split('-').str[0]
df['lz_ende'] = df['Laufzeit'].str.split('-').str[1]

Firmennamen, die in Komma enthalten werden neu formatiert. Die durch Kommata getrennten Namensbestandteile werden in umgekehrter Reihenfolge neu geordnet und in das neue Feld 'firmenname_neu' geschrieben.

In [3]:
def name_formatieren(firmenname):
    try:
        splitnamen = firmenname.split(', ')
        splitnamen.reverse()
        firmenname_neu = ' '.join(splitnamen).strip()
        #print(firmenname_neu)
    except:
        firmenname_neu = firmenname
    return firmenname_neu

df['firmenname_neu'] = df['Firmenname'].apply(name_formatieren)
df

,Firmenname,Ort,Sortiment,Verlag,Verweise/Bemerkungen/Ergänzungen/Zusätze,Lebensdaten,Laufzeit,Unnamed: 8,Unnamed: 9,lz_beginn,lz_ende,firmenname_neu
Nr,,,,,,,,,,,,
1,aar-Verlag,Wiesbaden,,x,Jolanda Debus,NaN,1973-1983,NaN,NaN,1973,1983,aar-Verlag
2,A4-Verlag,Essen,NaN,x,"Thomas Heidelbach, Dieter Sawatzki, Frank Schu...",NaN,1989-1998,NaN,NaN,1989,1998,A4-Verlag
3,Abakon Verlagsgesellschaft,München,NaN,x,Achim Schneider,NaN,1976-1987,NaN,NaN,1976,1987,Abakon Verlagsgesellschaft
4,ABB – Akademischer Buchvertrieb Bonn,"Bad Godesberg, Wesseling",x,NaN,"Dr. Rudolf Dünnhaupt, Boldt, Athenäum, Junker",NaN,1955-1986,NaN,NaN,1955,1986,ABB – Akademischer Buchvertrieb Bonn
5,ABB Arbeitsgemeinschaft Berliner Buchhändler,Berlin,x,NaN,Günter Wöckener-Guggisberg,21.06.1949-,1987-1990,NaN,NaN,1987,1990,ABB Arbeitsgemeinschaft Berliner Buchhändler
...,...,...,...,...,...,...,...,...,...,...,...,...
7137,Zwenka Verlag,Dortmund,NaN,x,Bernd Wagner,NaN,1978-1980,NaN,NaN,1978,1980,Zwenka Verlag
7138,Zwiebelzwerg Company Verlagsgesellschaft,Düsseldorf,NaN,x,"Gregor Schell, Dieter Herff",NaN,1981-1986,NaN,NaN,1981,1986,Zwiebelzwerg Company Verlagsgesellschaft
7139,Zwillings-Verlag,Pinneberg,NaN,x,"Reimer Lübbers, Peter Weidemann",NaN,1983-1985,NaN,NaN,1983,1985,Zwillings-Verlag


# GND Firmensuche

In [6]:
import requests
from bs4 import BeautifulSoup as soup
import unicodedata
from tqdm import tqdm
tqdm.pandas(desc="Fortschritt")

base_url = 'https://services.dnb.de/sru/authorities?'
params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
         }

#https://services.dnb.de/sru/authorities?recordSchema=MARC21-xml&operation=searchRetrieve&version=1.1&maximumRecords=50&accessToken=8090beb04b8a8ce79c2da2c1ee6890b8&query=KOE+%3D+Buchhandlung+Emil+Bierbaum+AND+BBG%3DTb*

#idn
#datafield tag="024" ind1="7" ind2=" ">
#<subfield code="a">2006064-6</subfield>

def gnd_firmen(row):
    # print(f'{row.firmenname_neu}')
    params.update({'query': f'KOE = {row.firmenname_neu} AND BBG=Tb*'})
    response = requests.get(base_url, params=params)
    response_xml = soup(response.content)
    
    try:
        matches = []
        for record in response_xml.find_all('record', {'type':'Authority'}):
            record_match = {}
            #idn bestimmen
            try:
                record_match['idn'] = record.find('controlfield', {'tag': '001'}).string.strip()
            except:
                record_match['idn'] = 'XX'
            
            #gnd-level bestimmten
            try:
                record_match['level'] = record.find('datafield', {'tag': '042'}).find('subfield', {'code':'a'}).string.strip()
            except:
                record_match['level'] = None

            #wirkungsort bestimmen
            #110g, 410g, 551a
            try:
                ortsliste = []
                try:
                    ort = record.find('datafield', {'tag':'551'}).find('subfield', {'code':'a'}).get_text(strip=True).strip()
                    ort = unicodedata.normalize("NFC",ort)
                    #print(f'try 1 {ort}')
                    ortsliste.append(ort)
                except:
                    None
                
                try:
                    ort = record.find('datafield', {'tag':'110'}).find('subfield', {'code':'g'}).get_text(strip=True).strip()
                    ort = unicodedata.normalize("NFC",ort)
                    #print(f'try 2 {ort}')
                    ortsliste.append(ort)
                except:
                    None
                
                try:
                    for fundstelle in record.find_all('datafield', {'tag':'410'}).find('subfield', {'code':'g'}).get_text(strip=True).strip():
                        ort = unicodedata.normalize("NFC",fundstelle)
                        #print(f'try 3 {ort}')
                        ortsliste.append(ort)
                except:
                    None
                #print(ortsliste)    
            except:
                ortsliste = []

            record_match['ort_match'] = True if str(row.Ort).strip() in ortsliste else False
            matches.append(record_match)
    except:
        matches = []
    
    #print(matches)
    return matches
        
df['firmen_matches'] = df.progress_apply(gnd_firmen, axis=1)



Fortschritt: 100%|██████████| 7135/7135 [25:03<00:00,  4.74it/s]


In [7]:
df[df.index.duplicated()]

,Firmenname,Ort,Sortiment,Verlag,Verweise/Bemerkungen/Ergänzungen/Zusätze,Lebensdaten,Laufzeit,Unnamed: 8,Unnamed: 9,lz_beginn,lz_ende,firmenname_neu,firmen_matches
Nr,,,,,,,,,,,,,


# GND-Auswertung

In [9]:
#es wird gezählt, wie oft im dictionary in matches True steht. wenn größer gleich 1, wird die idn zurückgegeben, hier muss nur ein Kriterium erfüllt werden, 
#weil bei der GND-Suche vorher auch nur ein Kriterium, nämlich der Verlagsort gegengecheckt wurde.


def gnd_auswertung(matches):
    try:
        idns = []
        for match in matches:
            treffer = 0
            for v in match.values():
                if isinstance(v, bool):
                    treffer += v
            if treffer >= 1:
                idns.append(match['idn'])
            else:
                None
    except:
        None
    return idns

df['idns'] = df.firmen_matches.apply(gnd_auswertung)
#df.matches.apply(gnd_auswertung)

In [10]:
df.to_csv('HABV_97_2_v2.csv')